### Combining primary data set (region-period with hunger scores) with ACLED conflict data aggregated at the ADMIN1 level.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import json
import requests

Read in the data of ineterest from ACLED's API.
Citation: " Armed Conflict Location & Event Data Project (ACLED); acleddata.com"

In [29]:
# request json and ignore metadata when reading into DataFrame

# must accept T&C
# limit default = 500
command = 'region=3&year>2008&limit=1000000'
r = requests.get(f"https://api.acleddata.com/acled/read?terms=accept&{command}")
acled = pd.DataFrame(r.json()['data'])

print(acled.shape)
acled.head()

(70041, 31)


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7099774,706,SOM31157,31157,2020-05-16,2020,1,Battles,Armed clash,Al Shabaab,...,Bilis Qooqaani,0.2837,41.7141,1,Radio Kulmiye,National,"On 16 May 2020, Al shabaab militants attacked ...",3,1589834547,SOM
1,7099905,800,UGA5984,5984,2020-05-15,2020,1,Battles,Armed clash,Pukwong Clan Militia (Uganda),...,Katum,3.5676,32.8161,2,URN (Uganda),National,"On 15 May 2020, Acholi clans (assumed Palaa, L...",0,1589834548,UGA
2,7099913,800,UGA5983,5983,2020-05-15,2020,1,Protests,Peaceful protest,Protesters (Uganda),...,Gulu,2.7667,32.3056,1,URN (Uganda),National,"On 15 May 2020, over 60 HIV positive residents...",0,1589834548,UGA
3,7099480,706,SOM31153,31153,2020-05-15,2020,1,Battles,Armed clash,Al Shabaab,...,Awdheegle,1.9833,44.8333,2,Undisclosed Source; Calamada,Local partner-Other,"On 15 May 2020, Al shabaab militants launched ...",0,1589834545,SOM
4,7099481,706,SOM31156,31156,2020-05-15,2020,1,Battles,Armed clash,Al Shabaab,...,Saabid,2.1311,45.0377,1,Undisclosed Source; Calamada,Local partner-Other,"On 15 May 2020, Al shabaab militants carried o...",0,1589834545,SOM


In [30]:
acled.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [31]:
acled[['country', 'admin1']].sample(5)

,country,admin1
43439,Somalia,Banadir
48674,Somalia,Banadir
68601,Uganda,Kitgum
10763,South Sudan,Central Equatoria
39434,Somalia,Middle Shabelle


In [32]:
acled = acled[['country', 'admin1', 'latitude', 'longitude', 'event_type', \
               'fatalities', 'event_date']]
acled = gpd.GeoDataFrame(acled, \
                         geometry = gpd.points_from_xy(acled.longitude.astype('float64'), \
                                                       acled.latitude.astype('float64')))
acled.head()

,country,admin1,latitude,longitude,event_type,fatalities,event_date,geometry
0,Somalia,Lower Juba,0.2837,41.7141,Battles,3,2020-05-16,POINT (41.71410 0.28370)
1,Uganda,Lamwo,3.5676,32.8161,Battles,0,2020-05-15,POINT (32.81610 3.56760)
2,Uganda,Gulu,2.7667,32.3056,Protests,0,2020-05-15,POINT (32.30560 2.76670)
3,Somalia,Lower Shabelle,1.9833,44.8333,Battles,0,2020-05-15,POINT (44.83330 1.98330)
4,Somalia,Lower Shabelle,2.1311,45.0377,Battles,0,2020-05-15,POINT (45.03770 2.13110)


In [33]:
acled['year'] = acled.event_date.map(lambda x: int(x.split('-')[0]))
acled['month'] = acled.event_date.map(lambda x: int(x.split('-')[1]))

In [34]:
acled.shape

(70041, 10)

In [35]:
acled_wide = acled.pivot(columns='event_type', values='fatalities')
acled_wide.shape

(70041, 6)

In [36]:
acled = acled.join(acled_wide)
acled.head()

,country,admin1,latitude,longitude,event_type,fatalities,event_date,geometry,year,month,Battles,Explosions/Remote violence,Protests,Riots,Strategic developments,Violence against civilians
0,Somalia,Lower Juba,0.2837,41.7141,Battles,3,2020-05-16,POINT (41.71410 0.28370),2020,5,3,NaN,NaN,NaN,NaN,NaN
1,Uganda,Lamwo,3.5676,32.8161,Battles,0,2020-05-15,POINT (32.81610 3.56760),2020,5,0,NaN,NaN,NaN,NaN,NaN
2,Uganda,Gulu,2.7667,32.3056,Protests,0,2020-05-15,POINT (32.30560 2.76670),2020,5,NaN,NaN,0,NaN,NaN,NaN
3,Somalia,Lower Shabelle,1.9833,44.8333,Battles,0,2020-05-15,POINT (44.83330 1.98330),2020,5,0,NaN,NaN,NaN,NaN,NaN
4,Somalia,Lower Shabelle,2.1311,45.0377,Battles,0,2020-05-15,POINT (45.03770 2.13110),2020,5,0,NaN,NaN,NaN,NaN,NaN


In [37]:
types = ['Battles', 'Protests', 'Riots', 'Explosions/Remote violence', 
         'Violence against civilians']

for vtype in types:
    acled[vtype] = pd.to_numeric(acled[vtype])

In [38]:
grouped = acled.groupby(by=['country', 'admin1', 'year', 'month'], sort=False)

In [39]:
acled_agg = pd.DataFrame()

In [40]:
grouped['Violence against civilians'].dtype

country      admin1             year  month
Somalia      Lower Juba         2020  5        float64
Uganda       Lamwo              2020  5        float64
             Gulu               2020  5        float64
Somalia      Lower Shabelle     2020  5        float64
South Sudan  Central Equatoria  2020  5        float64
                                                ...   
Uganda       Kamwenge           1997  1        float64
Rwanda       Southern Province  1997  1        float64
Burundi      Muramvya           1997  1        float64
Uganda       Gulu               1997  1        float64
             Mityana            1997  1        float64
Name: Violence against civilians, Length: 15529, dtype: object

In [41]:
# aggregate the conflict data at the admin1-month level by counting the number of 
# occurences and summing the number of fatalities for each conflict type

count_cols = ['battle_cnt', 'protest_cnt', 'riot_cnt', 'explosion_cnt', 'violence_on_civs_cnt']
acled_agg[count_cols] = grouped.count()[types]
acled_agg.head()

fatal_cols = ['battle_fatal', 'protest_fatal', 'riot_fatal', 'explosion_fatal', \
              'violence_on_civs_fatal']
acled_agg[fatal_cols] = grouped.sum()[types]

acled_agg.reset_index(inplace=True)

print(acled_agg.shape)
acled_agg.head()

(15529, 14)


,country,admin1,year,month,battle_cnt,protest_cnt,riot_cnt,explosion_cnt,violence_on_civs_cnt,battle_fatal,protest_fatal,riot_fatal,explosion_fatal,violence_on_civs_fatal
0,Somalia,Lower Juba,2020,5,7,0,0,1,1,9.0,0.0,0.0,2.0,0.0
1,Uganda,Lamwo,2020,5,1,0,0,0,3,0.0,0.0,0.0,0.0,0.0
2,Uganda,Gulu,2020,5,0,2,0,0,1,0.0,0.0,0.0,0.0,0.0
3,Somalia,Lower Shabelle,2020,5,24,0,0,6,3,46.0,0.0,0.0,5.0,3.0
4,South Sudan,Central Equatoria,2020,5,10,1,0,0,2,17.0,0.0,0.0,0.0,3.0


In [42]:
# write to disc
acled_agg.to_csv('conflict.csv')